# Importing libraries

In [1]:
import math
import os
import sys
from subprocess import call
from nltk import FreqDist
from nltk.util import ngrams
from nltk.stem.porter import PorterStemmer
import sklearn as sk
import pickle
import json
from collections import Counter
import requests
import matplotlib.pyplot as plt
import numpy as np

# Importing movie reviews dataset

In [6]:
# !wget https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json

In [7]:
with open("reviews.json", mode="r", encoding="utf-8") as f:
  reviews = json.load(f)

# Lexicon based approach

## Importing lexicon

In [4]:
# !wget https://gist.githubusercontent.com/bastings/d6f99dcb6c82231b94b013031356ba05/raw/f80a0281eba8621b122012c89c8b5e2200b39fd6/sent_lexicon

## Classification

In [8]:
# creating lexicon dictionary for easy access
lexicon_dict = {}

with open("sent_lexicon", mode="r", encoding="utf-8") as f:
  for i in f:
    i_list = i.strip().split()

    for j in i_list:
      key, value = j.split("=")

      if key == "word1":
        lexicon_dict[value] = 0
        word = value

      elif key == "priorpolarity":
        if value == "positive":
          lexicon_dict[word] = 1
        elif value == "negative":
          lexicon_dict[word] = -1

# creating function to classify reviews
def classify_review(review):
  # calculating binary score
  binary_score = 0
  for sentences in review["content"]:
    for word, _ in sentences:
      binary_score += lexicon_dict.get(word, 0)

  # classifying review
  THRESHOLD = 8
  if binary_score > THRESHOLD:
    return "POS"
  else:
    return "NEG"

# classifying reviews and checking accuracy
classifications = [classify_review(review) for review in reviews]
token_results = [1 if classification == reviews[i]["sentiment"] else 0 for i, classification in enumerate(classifications)]
token_accuracy = token_results.count(1)/len(token_results)
print("Accuracy: %0.2f" % token_accuracy)

Accuracy: 0.68
